#### Goal :: Predict the sale (number of items) for the shop-item pair in the  test set. The test set is for one month - November 2015 

In [424]:
from datetime import datetime, timedelta, date
from calendar import monthrange
from pytz import timezone
import pandas as pd
import numpy as np
import feather
import datetime
import pytz
import csv
import os
import matplotlib.pyplot as plt

%matplotlib inline


def get_dates_in_month(year, month, time_zone):
    num_days = monthrange(year, month)[1]
    first_date_of_month = datetime.datetime(year,month,1, tzinfo=time_zone)
    last_date_of_month =  datetime.datetime(year,month,num_days, tzinfo=time_zone)
    return get_dates_inrange(first_date_of_month, last_date_of_month)


def get_dates_inrange(date1, date2):
    if (not isinstance(date1, datetime.datetime)) | (not isinstance(date2, datetime.datetime)):
        return "date1 and date2 should be of type datetime.date"
    num_days = (date2 - date1).days + 1
    date_list = [date1 + datetime.timedelta(days=x) for x in range(0, num_days)]
    return date_list

#### Read data

In [425]:
DATA_FOLDER = './'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
date_df = pd.read_pickle("russian_holidays.pkl")
geography = pd.read_csv('geography.csv')
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))
holiday_filepath = 'hol.feather'
holidays = feather.read_dataframe(path)


In [ ]:
# print("Total Items: {}".format(items.item_id.nunique()))
# print("Total Categories: {}".format(item_categories.item_category_id.nunique()))
# print("Total Shops: {}".format(shops.shop_id.nunique()))
# print("\n--------------------------------------------------------\n")

# print("Training data")
# print("  # unique shops: {}".format(transactions.shop_id.nunique()))
# print("  # unique items: {}".format(transactions.item_id.nunique()))

# print("\n--------------------------------------------------------\n")
# print("Test data")
# print("  # unique shops: {}".format(test.shop_id.nunique()))
# print("  # unique items: {}".format(test.item_id.nunique()))


# %matplotlib inline


# unique_categories, count = np.unique(items.item_category_id, return_counts=True)
# ax = plt.subplot()
# ax.bar(unique_categories, count)    
# plt.title( "Distribution of Categories across all items")
# plt.show()

# unique_categories_trans, count_trans = np.unique(transactions.item_category_id, return_counts=True)
# ax = plt.subplot()
# ax.bar(unique_categories_trans, count_trans)
# plt.title("Distribution of Categories across all transactions")
# plt.show()

# unique_categories_test, count_test = np.unique(test_mod.item_category_id, return_counts=True)
# ax = plt.subplot()
# ax.bar(unique_categories_test, count_test)
# plt.title("Distribution of Categories across test dataset")
# plt.show()

# print("\n\n")

# items_test_nottrans = len(set(test.item_id.unique()) - set(test.item_id.unique()).intersection(set(transactions.item_id.unique())))
# intersection_items_test_transac = len(set(test.item_id.unique()).intersection(set(transactions.item_id.unique())))
# item_count_test = len(set(test.item_id.unique()))

# print("# items in test set: {}".format(item_count_test))
# print("# items in test set and transaction set: {}".format(intersection_items_test_transac))
# print("# items in test but not transaction: {}".format(items_test_nottrans))

# print("\n")

# shops_test_nottrans = len(set(test.shop_id.unique()) - set(test.shop_id.unique()).intersection(set(transactions.shop_id.unique())))
# intersection_shops_test_transac = len(set(test.shop_id.unique()).intersection(set(transactions.shop_id.unique())))
# shop_count_test = len(set(test.shop_id.unique()))

# print("# shops in test set: {}".format(shop_count_test))
# print("# shops in test set and transaction set: {}".format(intersection_shops_test_transac))
# print("# shops in test but not transaction: {}".format(shops_test_nottrans))

# print("\n")

# # Distribution of items across months

# #Get the items which are common to test and transaction data.
# #For each item:
# #    How many months have at least one entry for that item?

# # Items present in both test and transaction data


# item_distr = transactions.groupby(['item_id'])['date_block_num'].unique().reset_index(name='list_months')
# item_distr["num_months"] = item_distr.list_months.apply(lambda x: x.size)
# item_distr = item_distr.drop(['list_months'], axis=1)
# item_distr_testset = item_distr[item_distr.item_id.isin(list(s))]

###### Constructing previous_value_benchmark

For the each shop/item pair our predictions are just monthly sales from the previous month, i.e. October 2015.

The most important step at reproducing this score is correctly aggregating daily data and constructing monthly sales data frame. You need to get lagged values, fill NaNs with zeros and clip the values into [0,20] range. If you do it correctly, you'll get precisely 1.16777 on the public leaderboard.

Generating features like this is a necessary basis for more complex models. Also, if you decide to fit some model, don't forget to clip the target into [0,20] range, it makes a big difference.

In [ ]:
# def get_item_cnt(shopid, itemid):
#     if (shopid, itemid) in trans_oct_grouped:
#         count  = trans_oct_grouped[int(shopid), int(itemid)]
#         if(count > 20):
#             return 20
#         elif(count < 0):
#             return 0
#         else:
#             return count
#     else:
#         return 0

# trans_oct = transactions[transactions.date_block_num == 33]
# trans_oct_grouped = trans_oct.groupby(["shop_id", "item_id"]).item_cnt_day.sum()
# previous_value_benchmark_submission = test

# previous_value_benchmark_submission["item_cnt_month"] = previous_value_benchmark_submission.apply(lambda x: get_item_cnt(x.shop_id, x.item_id), axis=1)
# print(previous_value_benchmark_submission.item_cnt_month.describe())

# # writing previous_value_benchmark_submission ---> csv file
# previous_value_benchmark_submission[["ID", "item_cnt_month"]].to_csv("previous_value_benchmark_submission", index=False)

###### Preprocess transaction (train) and test dataset

There are some items in test set which do not exist in transaction dataset. 
However, these items will probabaly be similar to some other items in the transaction dataset.
We can use this simillarity to predict for these items

In [426]:
print(transactions.shape)
print(test.shape)

(2935849, 6)
(214200, 3)


In [427]:
# Drop the rows with price lt zero. ther is only one.
transactions = transactions[transactions.item_price > 0]

# Merging to get category_id in the transaction df
transactions = pd.merge(transactions, items[['item_id', 'item_category_id']], on='item_id', how='inner')

In [428]:
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,02.01.2013,0,59,22154,999.0,1.0,37
1,23.01.2013,0,24,22154,999.0,1.0,37
2,20.01.2013,0,27,22154,999.0,1.0,37
3,02.01.2013,0,25,22154,999.0,1.0,37
4,03.01.2013,0,25,22154,999.0,1.0,37


In [429]:
# Group the data by [date, shop_id, item_id ] to find the cumulative count. price is ignored for now
trans_a = transactions.groupby(['date', 'date_block_num', 'shop_id', 'item_id', 'item_category_id'])['item_cnt_day'].sum().reset_index(name='item_cnt')

In [430]:
trans_a.head(4)

,date,date_block_num,shop_id,item_id,item_category_id,item_cnt
0,01.01.2013,0,2,991,67,1.0
1,01.01.2013,0,2,1472,23,1.0
2,01.01.2013,0,2,1905,30,1.0
3,01.01.2013,0,2,2920,21,2.0


In [431]:
# looking at the price variability (in training data) of items which are in the test set.

trans_sub = transactions[transactions.item_id.isin(test.item_id.unique())]
print("total unique items in test set: {}".format(len(test.item_id.unique())))

num_const_price_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 1)
print(num_const_price_items)

price_change1_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 2)
print(price_change1_items)

price_change2_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 3)
print(price_change2_items)

price_change3_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 4)
print(price_change3_items)

price_change4_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 5)
print(price_change4_items)

price_change5_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 6)
print(price_change5_items)

print("\n")

print(num_const_price_items + price_change1_items + price_change2_items +
      price_change3_items + price_change4_items + price_change5_items)

total unique items in test set: 5100
797
584
491
413
361
315


2961


##### Building the shops data frame

In [432]:
# Setting the shop_type
shop_types = ['ТЦ', 'ТРК', 'ТРЦ', 'МТРЦ', 'ТК']
shops["shop_type"] = shops.shop_name.apply(lambda x: x.split()[1] if x.split()[1] in shop_types else 'OTH')

# Setting the city for each shop
shops["city"] = shops.shop_name.apply(lambda x: x.split()[0].replace("!", ""))

# Appending the geographic info to shops df
shops = pd.merge(shops, geography.drop(['city_eng'], axis=1), on='city', how='inner')

print(shops.shape)
shops.head()

(60, 8)


,shop_name,shop_id,shop_type,city,time_wrt_utc,fed_subject,fed_dist,dist_from_moscow
0,"!Якутск Орджоникидзе, 56 фран",0,OTH,Якутск,9,Sakha Republic,Far East,8395
1,"!Якутск ТЦ ""Центральный"" фран",1,ТЦ,Якутск,9,Sakha Republic,Far East,8395
2,"Якутск Орджоникидзе, 56",57,OTH,Якутск,9,Sakha Republic,Far East,8395
3,"Якутск ТЦ ""Центральный""",58,ТЦ,Якутск,9,Sakha Republic,Far East,8395
4,"Адыгея ТЦ ""Мега""",2,ТЦ,Адыгея,3,Adygea Republic,South,1589


##### Building the transaction data frame

In [433]:
# Appending temporal information

# trans_a['day'] = trans_a.date.apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").day)
# trans_a['month'] = trans_a.date.apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").month)
# trans_a['year'] = trans_a.date.apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").year)
# trans_a = pd.merge(trans_a, date_df, on= (['year', 'month', 'day']), how='inner')

# Appending shop information
trans_a = pd.merge(trans_a, shops, on='shop_id', how='inner')

In [434]:
trans_a.head(3)

,date,date_block_num,shop_id,item_id,item_category_id,item_cnt,shop_name,shop_type,city,time_wrt_utc,fed_subject,fed_dist,dist_from_moscow
0,01.01.2013,0,2,991,67,1.0,"Адыгея ТЦ ""Мега""",ТЦ,Адыгея,3,Adygea Republic,South,1589
1,01.01.2013,0,2,1472,23,1.0,"Адыгея ТЦ ""Мега""",ТЦ,Адыгея,3,Adygea Republic,South,1589
2,01.01.2013,0,2,1905,30,1.0,"Адыгея ТЦ ""Мега""",ТЦ,Адыгея,3,Adygea Republic,South,1589


## TO Do: 

### Get the preliminary feature set (done)
Get rid of some unnecesary columns

### Feature preprocessing:
check for NAN values...
encoding


 prepare test set also. ( cross between dates and input test file(item-shop pair)). 
Additionally append the shop, temporal columns to test.

 How are we training?


Ideally there should be training at the item level. i.e. one model for each item.
 However, that is not possible because there are some items which are not in training data.

#### Build the training set

In [302]:
### incomplete...

#### Build the test set

In [435]:
from itertools import product

year=2015
month=11
moscow = timezone('Europe/Moscow')

# get the list of dates in Nov-2015
date_list = get_dates_in_month(year, month, moscow)

index_cols = ['shop_id', 'item_id', 'date']

# grid will have all cominates of shop_ids, item_ids and dates (shape: 6426000 rows, 3 columns)
grid = []
test_items = test.item_id.unique()
test_shops = test.shop_id.unique()
for d in date_list:
    grid.append(np.array(list(product(*[test_shops, test_items, [d]]))))

# Turn the grid into a dataframe
index_cols = ['shop_id', 'item_id', 'date']
grid = pd.DataFrame(np.vstack(grid), columns = index_cols)

test_df = grid

test_df['weekday'] = test_df['date'].apply(lambda x: x.isoweekday())
test_df['isweekend'] = test_df['date'].apply(lambda x: True if (x.isoweekday()==6 | x.isoweekday()==7) else False )
test_df['date_str'] = test_df.date.apply(lambda x: x.isoformat().split("T")[0])

# Appending category_id
test_df = pd.merge(test_df, items[['item_id', 'item_category_id']], on='item_id', how='inner')

holidays_b = holidays
holidays_b['date_str'] = holidays_b.date.apply(lambda x: x.isoformat().split("T")[0])
holidays_b['off_day'] =  holidays_b.h_type.apply(lambda x: x.split(",")[0] == 'National holiday')
holidays_b['isholiday'] = True

# Appending isholiday
test_df = pd.merge(test_df, holidays_b[['date_str', 'isholiday']], on='date_str', how='left')
test_df['isholiday'].fillna(False, inplace=True)

# off_days - sunday, saturday, holidays-off_days
test_df = pd.merge(test_df, holidays_b[['off_day', 'date_str']], on='date_str', how='left')
test_df.loc[test_df.loc[:, 'weekday'] >= 6, 'off_day'] = True


In [436]:
print(test_df.shape)
test_df[test_df.date_str == '2015-11-04'].head(3)

(6426000, 9)


,shop_id,item_id,date,weekday,isweekend,date_str,item_category_id,isholiday,off_day
126,5,5037,2015-11-04 00:00:00+02:30,3,False,2015-11-04,19,True,True
127,4,5037,2015-11-04 00:00:00+02:30,3,False,2015-11-04,19,True,True
128,6,5037,2015-11-04 00:00:00+02:30,3,False,2015-11-04,19,True,True
